In [132]:
import json
from collections import defaultdict, deque

class C3PO:
    def __init__(self, millenniumFalconJsonFilePath):
        with open(millenniumFalconJsonFilePath, 'r') as f:
            self.millennium_falcon_data = json.load(f)
        self.autonomy = self.millennium_falcon_data["autonomy"]
        self.routes = self.millennium_falcon_data["routes"]
        
        self.graph = defaultdict(list)
        for route in self.routes:
            origin = route["origin"]
            destination = route["destination"]
            travelTime = route["travelTime"]
            self.graph[origin].append((destination, travelTime))
            self.graph[destination].append((origin, travelTime))

        for node in self.graph:
            self.graph[node].append((node, 0))
    
    def giveMeTheOdds(self, empireJsonFilePath):
        with open(empireJsonFilePath, 'r') as f:
            self.empire_data = json.load(f)
        countdown = self.empire_data["countdown"]
        bounty_hunters = self.empire_data["bounty_hunters"]
        
        self.bounty_hunters_plan = defaultdict(list)
        for bh in bounty_hunters:
            planet = bh["planet"]
            day = bh["day"]
            self.bounty_hunters_plan[planet].append(day)
        
        start_planet = "Tatooine"
        end_planet = "Endor"
 
        dp = defaultdict(lambda: float('inf'))
        dp[(start_planet, 0)] = 0.0
        
        # Queue with (current planet, current day, current capture encounters, remaining autonomy)
        queue = deque([(start_planet, 0, 0, self.autonomy)])

        way_to_endor = []
        while queue:
            # print('Queue is:', list(queue))
            current_planet, current_day, current_encounters, remaining_autonomy = queue.popleft()
            # print(f"Exploring: {current_planet} on day {current_day} with {current_encounters} encounters and {remaining_autonomy} autonomy left")
  
            if current_planet == end_planet and current_day <= countdown:
                way_to_endor.append((current_planet, current_day, current_encounters, remaining_autonomy))
                # print(f"Reached Endor on day {current_day} with probability {round(1 - current_probability, 2)}")
                continue
                    
            for neighbor, travel_time in self.graph[current_planet]:
                # print(f"Considering move to: {neighbor} with travel time: {travel_time}")
                if current_day + travel_time <= countdown:
                    # print(f"Can reach {neighbor} by day {current_day + travel_time} within countdown {countdown}")
                    new_day = current_day + travel_time
                    new_encounters = current_encounters
                    new_remaining_autonomy = remaining_autonomy - travel_time
                    
                    if neighbor in self.bounty_hunters_plan and new_day in self.bounty_hunters_plan[neighbor]:
                        new_encounters += 1
                        # print(f"Encounter with bounty hunters on {neighbor} on day {new_day}. Total encounters: {new_encounters}")
                    
                    if neighbor == current_planet:
                        new_day += 1
                        new_remaining_autonomy = self.autonomy
                        # print(f"Refueling on the same planet {current_planet}, reset autonomy to {self.autonomy}")
                    
                    if new_remaining_autonomy == 0:
                        # print(f"No more fuel")
                        new_day += 1
                        new_remaining_autonomy = self.autonomy
                        if neighbor in self.bounty_hunters_plan and (new_day) in self.bounty_hunters_plan[neighbor]:
                            new_encounters += 1
                            # print(f"Encounter with bounty hunters during refuel on {neighbor} on day {new_day}. Total encounters: {new_encounters}")
                    
                    if dp[(neighbor, new_day)] >= new_encounters:
                        dp[(neighbor, new_day)] = new_encounters
                        queue.append((neighbor, new_day, new_encounters, new_remaining_autonomy))
                        # print(f"Added to queue: {neighbor} on day {new_day} with {new_encounters} encounters and {new_remaining_autonomy} autonomy left")
                        # print(neighbor, new_day, new_encounters, new_remaining_autonomy)

        if not way_to_endor:
            return 0.0
        else : 
            best_path = min(way_to_endor, key=lambda x: x[2])
            return 1 - round(1 - (0.9 ** best_path[2]), 2)


In [133]:
example_directory = "examples/"

number_of_examples = 4
expected_result = [0, 0.81, 0.9, 1]

for i in range(1, number_of_examples + 1):
    example_sub_dir = example_directory + "example" + str(i) + "/" 
    millennium_falcon_json_path = example_sub_dir + "millennium-falcon.json"
    empire_json_path = example_sub_dir + "empire.json"
    c3po = C3PO(millennium_falcon_json_path)
    odds = c3po.giveMeTheOdds(empire_json_path)
    if odds == expected_result[i - 1]:
        print("Example", i, ":", odds, "Correct")
    else : 
        print("Example", i, ":", odds, "Incorrect")
    print()


Example 1 : 0.0 Correct

Example 2 : 0.81 Correct

Example 3 : 0.9 Correct

Example 4 : 1.0 Correct

